In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report
from sklearn import preprocessing
# 数据是否需要标准化
scale = True #True代表作标准化

In [ ]:
# 载入数据
data = np.genfromtxt("LR-testSet.csv", delimiter=",")
x_data = data[:,:-1]
y_data = data[:,-1]
#画图，这个是二元分类问题
def plot():
    x0 = []
    x1 = []
    y0 = []
    y1 = []
    # 切分不同类别的数据
    for i in range(len(x_data)):
        if y_data[i]==0:
            x0.append(x_data[i,0])
            y0.append(x_data[i,1])
        else:
            x1.append(x_data[i,0])
            y1.append(x_data[i,1])

    # 画图
    scatter0 = plt.scatter(x0, y0, c='b', marker='^')
    scatter1 = plt.scatter(x1, y1, c='r', marker='x')
    #画图例
    plt.legend(handles=[scatter0,scatter1],labels=['label0','label1'],loc='best')
    
plot()
plt.show()

In [ ]:
# 数据处理，添加偏置项
x_data = data[:,:-1]
y_data = data[:,-1,np.newaxis]#newaxis加上一个维度

print(np.mat(x_data).shape)
print(np.mat(y_data).shape)
# 给样本添加偏置项
X_data = np.concatenate((np.ones((100,1)),x_data),axis=1)#生成100行1列的numpyArray，里面的值都是1,然后和x_data合并
print(X_data.shape)
# print(x_data)
# print(X_data)#第0列都是1，其他两列是特征值

In [ ]:
def sigmoid(x):
    return 1.0/(1+np.exp(-x))

def cost(xMat, yMat, ws):#逻辑回归的代价函数，传入两个矩阵
    left = np.multiply(yMat, np.log(sigmoid(xMat*ws))) #数据矩阵*权值矩阵=预测值，multiply是对位相乘，不是矩阵相乘，叫按位相乘，sigmoid是决策边界函数
    right = np.multiply(1 - yMat, np.log(1 - sigmoid(xMat*ws)))
    return np.sum(left + right) / -(len(xMat))

def gradAscent(xArr, yArr):
    
    if scale == True:
        xArr = preprocessing.scale(xArr)
    xMat = np.mat(xArr) #这里转换为矩阵
    yMat = np.mat(yArr)
    lr = 0.001
    epochs = 10000
    costList = []
    # 计算数据行列数
    # 行代表数据个数，列代表权值个数
    m,n = np.shape(xMat)
    # 初始化权值,也是一个矩阵，初值为1 ，三行一列
    ws = np.mat(np.ones((n,1))) 
    
    for i in range(epochs+1):             
        # xMat和weights矩阵相乘
        h = sigmoid(xMat*ws)   #100个样本的输出值
        # 计算误差
        ws_grad = xMat.T*(h - yMat)/m #在这里已经把累加做了
        ws = ws - lr*ws_grad 
        
        if i % 50 == 0:
            costList.append(cost(xMat,yMat,ws))#训练数据，标签，权值
    return ws,costList

In [ ]:
# 训练模型，得到权值和cost值的变化
gradAscent(X_data, y_data)
ws,costList = gradAscent(X_data, y_data)
print(ws)

In [ ]:
if scale == False:
    # 画图决策边界
    plot()
    x_test = [[-4],[3]]
    y_test = (-ws[0] - x_test*ws[1])/ws[2]
    plt.plot(x_test, y_test, 'k')
    plt.show()

In [ ]:
# 画图 loss值的变化
x = np.linspace(0,10000,201)
plt.plot(x, costList, c='r')
plt.title('Train')
plt.xlabel('Epochs')
plt.ylabel('Cost')
plt.show()

In [ ]:
# 预测
def predict(x_data, ws):
    if scale == True:
        x_data = preprocessing.scale(x_data)
    xMat = np.mat(x_data)
    ws = np.mat(ws)
    return [1 if x >= 0.5 else 0 for x in sigmoid(xMat*ws)]

predictions = predict(X_data, ws)

print(classification_report(y_data, predictions)) #sklearn里面用来计算准确率召回率的方法